# Master-Detail Adapter

> Adapter for integrating cjm-fasthtml-interactions MasterDetail pattern into settings

In [ ]:
#| default_exp components.master_detail_adapter

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Union, Optional, Any
from pathlib import Path
from fasthtml.common import *

from cjm_fasthtml_interactions.patterns.master_detail import MasterDetail, DetailItem, DetailItemGroup
from cjm_fasthtml_interactions.core.context import InteractionContext
from cjm_fasthtml_daisyui.components.data_display.badge import badge_colors

from cjm_fasthtml_settings.core.utils import (
    load_config,
    get_default_values_from_schema,
)
from cjm_fasthtml_settings.components.forms import create_settings_form_container

## Settings Detail Renderer

This function renders the detail content for a settings schema using the standard settings form.

In [ ]:
#| export
def create_settings_detail_renderer(
    config_dir: Path,  # Configuration directory
    save_route_fn: callable,  # Function that returns save route URL for schema_id
    reset_route_fn: callable  # Function that returns reset route URL for schema_id
) -> callable:  # Render function for detail view
    """Create a render function for settings detail view.
    
    This creates a closure that captures the config_dir and route functions,
    returning a render function compatible with DetailItem.
    """
    def render_settings_detail(ctx: InteractionContext):
        """Render settings form for a schema."""
        schema = ctx.get_data("schema", {})
        schema_id = ctx.get_data("schema_id", "")
        
        # Load existing config or use defaults
        saved_config = load_config(schema_id, config_dir)
        default_values = get_default_values_from_schema(schema)
        values = {**default_values, **saved_config}
        
        return create_settings_form_container(
            schema=schema,
            values=values,
            post_url=save_route_fn(schema_id),
            reset_url=reset_route_fn(schema_id),
            use_alert_container=True
        )
    
    return render_settings_detail

## Settings Data Loader

This function creates a data loader that provides schema information to the render function.

In [ ]:
#| export
def create_settings_data_loader(
    schema: Dict,  # JSON schema
    schema_id: str  # Schema identifier
) -> callable:  # Data loader function
    """Create a data loader that provides schema information."""
    def load_schema_data(request):
        """Load schema data for rendering."""
        return {
            "schema": schema,
            "schema_id": schema_id
        }
    
    return load_schema_data

## Configuration Badge Helper

Check if a schema has been configured (config file exists).

In [ ]:
#| export
def is_schema_configured(
    schema_id: str,  # Schema identifier
    config_dir: Path  # Configuration directory
) -> bool:  # True if config file exists
    """Check if a schema has been configured."""
    config_file = config_dir / f"{schema_id}.json"
    return config_file.exists()

## Settings Master-Detail Creator

This is the main adapter function that creates a `MasterDetail` instance configured for settings.

In [ ]:
#| export
def create_settings_master_detail(
    schemas: Dict,  # All registered schemas (from registry.get_all())
    config_dir: Path,  # Configuration directory
    save_route_fn: callable,  # Function that returns save route URL for schema_id
    reset_route_fn: callable,  # Function that returns reset route URL for schema_id
    default_schema: str = "general",  # Default schema to show
    menu_section_title: str = "Settings",  # Title for master list
    plugin_registry: Optional[Any] = None  # Optional plugin registry
) -> MasterDetail:  # Configured MasterDetail instance
    """Create a MasterDetail instance configured for settings.
    
    This adapter function transforms the settings schema structure into
    DetailItem and DetailItemGroup objects compatible with MasterDetail.
    """
    from cjm_fasthtml_settings.core.schema_group import SchemaGroup
    
    # Create the settings detail renderer
    render_fn = create_settings_detail_renderer(config_dir, save_route_fn, reset_route_fn)
    
    # Convert schemas to DetailItems and DetailItemGroups
    items = []
    
    for schema_entry in schemas.values():
        if isinstance(schema_entry, SchemaGroup):
            # Handle schema groups
            group_items = []
            configured_count = 0
            
            for sub_schema in schema_entry.schemas.values():
                schema_id = sub_schema.get("unique_id", sub_schema.get("name"))
                configured = is_schema_configured(schema_id, config_dir)
                if configured:
                    configured_count += 1
                
                group_items.append(
                    DetailItem(
                        id=schema_id,
                        label=sub_schema.get("menu_title", sub_schema.get("title", schema_id)),
                        render=render_fn,
                        data_loader=create_settings_data_loader(sub_schema, schema_id),
                        badge_text="configured" if configured else None,
                        badge_color=badge_colors.success if configured else None
                    )
                )
            
            items.append(
                DetailItemGroup(
                    id=schema_entry.name,
                    title=schema_entry.title,
                    items=group_items,
                    default_open=schema_entry.default_open,
                    badge_text=f"{configured_count} configured" if configured_count > 0 else None,
                    badge_color=badge_colors.success if configured_count > 0 else None
                )
            )
        else:
            # Handle individual schemas
            schema = schema_entry
            schema_id = schema.get("unique_id", schema.get("name"))
            configured = is_schema_configured(schema_id, config_dir)
            
            items.append(
                DetailItem(
                    id=schema_id,
                    label=schema.get("menu_title", schema.get("title", schema_id)),
                    render=render_fn,
                    data_loader=create_settings_data_loader(schema, schema_id),
                    badge_text="configured" if configured else None,
                    badge_color=badge_colors.success if configured else None
                )
            )
    
    # Add plugin items if registry is provided
    if plugin_registry:
        # Get all categories
        categories_with_plugins = plugin_registry.get_categories_with_plugins()
        
        for category in categories_with_plugins:
            # Get plugins for this category
            category_plugins = plugin_registry.get_plugins_by_category(category)
            plugin_items = []
            
            for plugin_metadata in category_plugins:
                if plugin_metadata.config_schema:
                    # Use 'name' attribute from PluginMetadata
                    plugin_id = plugin_metadata.name
                    
                    # Check if plugin is configured
                    try:
                        config_data = plugin_registry.load_plugin_config(plugin_id)
                        configured = bool(config_data)
                    except:
                        configured = False
                    
                    plugin_items.append(
                        DetailItem(
                            id=plugin_id,
                            label=plugin_metadata.title,
                            render=render_fn,
                            data_loader=create_settings_data_loader(
                                plugin_metadata.config_schema,
                                plugin_id
                            ),
                            badge_text="configured" if configured else None,
                            badge_color=badge_colors.success if configured else None
                        )
                    )
            
            if plugin_items:
                configured_count = sum(1 for item in plugin_items if item.badge_text == "configured")
                display_name = plugin_registry.get_category_display_name(category)
                
                items.append(
                    DetailItemGroup(
                        id=f"plugins-{category.lower().replace(' ', '-')}",
                        title=display_name or f"{category.title()} Plugins",
                        items=plugin_items,
                        default_open=False,
                        badge_text=f"{configured_count}/{len(plugin_items)}" if configured_count > 0 else None,
                        badge_color=badge_colors.info
                    )
                )
    
    # Create and return the MasterDetail instance
    return MasterDetail(
        interface_id="settings",
        items=items,
        default_item=default_schema,
        master_title=menu_section_title,
        master_width="w-64"
    )

## Usage Example

In [ ]:
# Example: Create settings master-detail
from pathlib import Path
from cjm_fasthtml_settings.core.schemas import registry
from cjm_fasthtml_settings.core.config import get_app_config_schema

# Register a sample schema
registry.register(get_app_config_schema(app_title="Test App", include_theme=False))

# Create the master-detail interface
settings_md = create_settings_master_detail(
    schemas=registry.get_all(),
    config_dir=Path("configs"),
    save_route_fn=lambda id: f"/settings/save?id={id}",
    reset_route_fn=lambda id: f"/settings/reset?id={id}",
    default_schema="general",
    menu_section_title="Settings"
)

print(f"Settings master-detail created with {len(settings_md.item_index)} items")
print(f"Items: {list(settings_md.item_index.keys())}")

Settings master-detail created with 1 items
Items: ['general']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()